## Языковое моделирование

Языковые модели — важнейшая часть современного NLP. Практически во всех задачах, связанных с обработкой текста, напрямую или косвенно используются языковые модели. А наиболее известные недавние прорывы в области - это по большей части новые подходы к языковому моделированию. ELMO, BERT, GPT - это языковые модели.

In [6]:
# !pip install razdel

In [33]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 

Возьмём новостной корпус, собранный с lenta.ru, и файл с "Мастером и Маргаритой" Булгакова.

In [16]:
news = open('lenta.txt', encoding="utf-8").read()
mim = open('the_master_and_margarita.txt').read()

Напишем простую функцию для нормализации. Удалять пунктуацию и приводить к нижнему регистру, строго говоря не стоит, сгенерированный текст так будет не похож на настоящий. Но это немного упростит нам работу. 

In [6]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


Посмотрим на количество токенов в тексте.

In [17]:
norm_news = normalize(news)
norm_mim = normalize(mim)

In [19]:
print("Длина корпуса новостных текстов в токенах - ", len(norm_news))
print("Длина корпуса \"Мастера и Маргариты\" в токенах - ", len(norm_mim))

Длина корпуса новостных текстов в токенах -  1505789
Длина корпуса "Мастера и Маргариты" в токенах -  119394


И по уникальным токенам

In [20]:
print("Уникальный токенов в корпусе новостных текстов - ", len(set(norm_news)))
print("Уникальный токенов в корпусе \"Мастера и Маргариты\" - ", len(set(norm_mim)))

Уникальный токенов в корпусе новостных текстов -  116302
Уникальный токенов в корпусе "Мастера и Маргариты" -  23900


Посчитаем, сколько раз встречаются слова и выведем самые частотные.

In [10]:
from collections import Counter

In [22]:
vocab_news = Counter(norm_news)
vocab_mim = Counter(norm_mim)

In [23]:
vocab_mim.most_common(10)

[('–', 5344),
 ('и', 5016),
 ('в', 3645),
 ('не', 2026),
 ('на', 2003),
 ('что', 1750),
 ('с', 1292),
 ('он', 1151),
 ('а', 970),
 ('я', 863)]

In [12]:
vocab_news.most_common(10)

[('в', 72412),
 ('и', 33290),
 ('на', 28434),
 ('по', 19490),
 ('что', 17031),
 ('с', 15921),
 ('не', 12702),
 ('из', 7727),
 ('о', 7515),
 ('как', 7514)]

Для того, чтобы превратить абсолютные частоты в вероятности, разделим на общее число слов в каждом корпусе.

In [13]:
vocab_news['ваываываываываыва']

0

In [14]:
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})
probas_news.most_common(20)

[('в', 0.04808907489694771),
 ('и', 0.0221080111489724),
 ('на', 0.018883123731146926),
 ('по', 0.012943380513471676),
 ('что', 0.011310349590812525),
 ('с', 0.01057319451795703),
 ('не', 0.008435444806676101),
 ('из', 0.005131529052211166),
 ('о', 0.00499073907433246),
 ('как', 0.0049900749706632205),
 ('к', 0.00407161959610543),
 ('за', 0.0040125143695431435),
 ('россии', 0.0036751497055696383),
 ('для', 0.003325831175549828),
 ('его', 0.003260084912295149),
 ('он', 0.0031704309169478593),
 ('от', 0.003066830744546547),
 ('сообщает', 0.003050228152815567),
 ('а', 0.0029180715226369697),
 ('также', 0.002716184007188258)]

In [24]:
probas_mim = Counter({word:c/len(norm_mim) for word, c in vocab_mim.items()})
probas_mim.most_common(20)

[('–', 0.04475936814245272),
 ('и', 0.04201216141514649),
 ('в', 0.030529172320217096),
 ('не', 0.016969026919275675),
 ('на', 0.016776387423153592),
 ('что', 0.014657352965810676),
 ('с', 0.010821314303901368),
 ('он', 0.009640350436370336),
 ('а', 0.008124361358192203),
 ('я', 0.007228168919711208),
 ('как', 0.007035529423589125),
 ('но', 0.005946697488986046),
 ('к', 0.005854565556058094),
 ('его', 0.005770809253396318),
 ('это', 0.005393905891418329),
 ('же', 0.004975124378109453),
 ('…', 0.004631723537196174),
 ('из', 0.004422332780541736),
 ('у', 0.004422332780541736),
 ('по', 0.003978424376434327)]

Эти вероятности уже можно использовать, чтобы ответить на вопрос — это предложение больше подходит для новостей или для текста авторства Булгакова?

В теории вероятностей для того, чтобы найти общую вероятность нескольких независимых событий произойти одновременно, нужно перемножить вероятности отдельных событий. В нашем случае мы хотим найти вероятность получить данное предложение. Для этого мы перемножаем вероятности слов в этом предложении. 

(Если бы мы сложили вероятности, то мы бы получили вероятность выбрать из корпуса 1 из слов в данном предложении)

Напишем простую функцию, которая расчитает общую вероятность. Вместо умножения вероятностей можно складывать логарифмы от них. Еще нам нужно учесть одну деталь - некоторых слов может не быть в словаре и, соответственно, вероятность будет нулевая. Можно использовать в таких случаях небольшое значение вероятности, например 1/длина корпуса. Исправить это по-нормальному - сложно, придется подробнее разбираться с вероятностями, сглаживаниями и заменой неизвестных слов.

In [25]:
def compute_joint_proba(text, word_probas):
    prob = 0
    for word in normalize(text):
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += (np.log(1/len(norm_mim)))
    
    return np.exp(prob)

In [26]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'

Расчитаем вероятность встретить такой текст в каждом из корпусов (для таких маленьких чисел нужно смотреть на степень после e: чем больше степень, тем больше вероятность; но тут легко запутаться так как степень будет отрицательная и больше будет число, которое ближе к нулю (-5 больше -10 например)

In [27]:
compute_joint_proba(phrase, probas_mim)

8.390252792918053e-47

In [28]:
compute_joint_proba(phrase, probas_news)

4.573351371331133e-45

Можно просто довериться функции больше/меньше, чтобы не запутаться

In [29]:
compute_joint_proba(phrase, probas_news) > compute_joint_proba(phrase, probas_mim)

True

Вероятность встретить такой текст в новостном корпусе выше. Попробуем другой текст:

In [30]:
phrase = 'Но дни и в мирные и в кровавые годы летят как стрела, и молодые Турбины не заметили, как в крепком морозе наступил белый, мохнатый декабрь.'
compute_joint_proba(phrase, probas_news) > compute_joint_proba(phrase, probas_mim)

False

Тут получается обратная ситуация.

Однако такая оценка вероятности предложения чересчур упрощает действительность. Слова в предложении - это не независимые события, выбор первого слова сильно влияет на вероятности выбрать второе, третье и так далее.

Такие события можно оценивать по формуле полной вероятности:

In [73]:
Image(url="https://i.ibb.co/sC7CKzQ/image.png",
     width=500, height=500)

А если простыми словами, то для того, чтобы получить вероятность предложения, нужно перемножить вероятность первого слова, вероятность второго слова, при условии первого, вероятность третьего при условии первого и второго, вероятность четвертого слова, при условии первого, второго и третьего и так далее до вероятности последнего слова при условии всех предшествующих.

Условные вероятности для слов можно также вычислить по частотностям. Вероятность слова А при условии слова Б равна отношению количества раз, которое встретились слова А и Б вместе, к количеству раз, которое встретилось слово Б. Вероятность слова В при условии А и Б равна отношению количества раз, которое встретились слова А,Б и В вместе к количеству раз, которое встретились слова А и Б.
И так далее. 

Но тут появляется проблема. Для того, чтобы расчитать полную вероятность предложения нужно, чтобы такое предложение уже встретилось в корпусе хотя бы 1 раз. Очевидно, что даже огромный корпус всего написанного текста не включает в себя все возможные тексты (тем более маленький корпус). Поэтому один из множителей в произведении будет нулевым, а значит и все произведение станет нулевым.

Для того, чтобы этого избежать можно поубавить строгости и предположить, что вероятность слова зависит только от предыдущего слова. Это предположение называется марковским (в честь математика Андрея Маркова). Такую модель еще можно назвать биграммной.

Чтобы расчитать вероятность с таким предположением, нам достаточно найти количество вхождений для каждого биграмма. А частоты отдельных слов у нас уже есть. 

In [34]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

Вероятность первого слово можно по идее считать просто как вероятность униграмма, но можно сделать небольшое добавление в нашу модель - поставить в начала каждого предложения технический токен начала предложения, а вероятность первого слова рассчитывать как вероятность биграма старт-первое слово поделить на частоту старта. Дальше мы будем генерировать текст с помощью языковой модели и это поможет нам генерировать более красивые предложения.


Для того, чтобы у нас получились честные вероятности и можно было посчитать вероятность первого слова, нам нужно добавить тэг маркирующий начало предложений \< start \>

Дальше мы попробуем сгенерировать текст, используя эти вероятности, и нам нужно будет когда-то остановится. Для этого добавим тэг окончания \< end \>

Ну и поделим все на предложения

In [35]:
sentences_mim = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(mim[:5000000])]
sentences_news = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [36]:
unigrams_mim = Counter()
bigrams_mim = Counter()

for sentence in sentences_mim:
    unigrams_mim.update(sentence)
    bigrams_mim.update(ngrammer(sentence))


unigrams_news = Counter()
bigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))


Чтобы посчитать условную вероятность мы можем поделить количество вхождений на количество вхождений первого слова. Обновим нашу функцию

In [37]:
def compute_joint_proba_markov_assumption(text, word_counts, bigram_counts):
    prob = 0
    for ngram in ngrammer(['<start>'] + normalize(phrase) + ['<end>']):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)
    
    return np.exp(prob)

In [40]:
# Эта фраза более вероятна в корпусе Булгакова
phrase = 'Когда отпевали мать, был май, вишневые деревья и акации наглухо залепили стрельчатые окна.'

compute_joint_proba_markov_assumption(phrase, unigrams_mim, bigrams_mim) > \
compute_joint_proba_markov_assumption(phrase, unigrams_news, bigrams_news)

True

In [39]:
# Эта фраза более вероятна в корпусе новостей
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'

compute_joint_proba_markov_assumption(phrase, unigrams_mim, bigrams_mim) > \
compute_joint_proba_markov_assumption(phrase, unigrams_news, bigrams_news)

False

### Генерация текста

Попробуем теперь генерировать текст с помощью биграмной языковой модели. Принцип генерации очень простой - на каждом шаге мы случайно выбираем следующее слово согласно вероятностям, расчитанным по 1 предыдущему слову.

При генерации мы можем выбирать только из уже известных слов. Можно заранее рассчитать все вероятности и сохранить их в матрицу. Размерность матрицы слова на слова. В каждой ячейке будет лежать вероятность получить слово Б, после слова А. Слово А будет в строке, а Б в колонке.

Матрицы получатся очень большими, но большинство значений будет нулевыми, поэтому можно воспользоваться разреженным форматом.

In [ ]:
from scipy.sparse import lil_matrix

In [47]:
# матрица слова на слова (инициализируем нулями)
matrix_mim = lil_matrix((len(unigrams_mim), 
                         len(unigrams_mim)))

# к матрице нужно обращаться по индексам
# поэтому зафиксируем порядок слов в словаре и сделаем маппинг id-слово и слово-id
id2word_mim = list(unigrams_mim)
word2id_mim = {word:i for i, word in enumerate(id2word_mim)}

# заполняем матрицу
for ngram in bigrams_mim:
    word1, word2 = ngram.split()
    # на пересечение двух слов ставим вероятность встретить второе после первого
    matrix_mim[word2id_mim[word1], word2id_mim[word2]] =  (bigrams_mim[ngram]/
                                                                     unigrams_mim[word1])

In [48]:
# то же самое для другого корпуса
matrix_news = lil_matrix((len(unigrams_news), 
                        len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}



for ngram in bigrams_news:
    word1, word2 = ngram.split()
    matrix_news[word2id_news[word1], word2id_news[word2]] =  (bigrams_news[ngram]/
                                                                     unigrams_news[word1])


Для генерации нам понадобится функция np.random.choice , которая выбирает случайный объект из заданных. Ещё в неё можно подать вероятность каждого объекта и она будет доставать по ним (не только максимальный по вероятности)

In [49]:
def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        # просто выбирать наиболее вероятное продолжение не получится
        # можете попробовать раскоментировать следующую строчку и посмотреть что получается
#         chosen = matrix[current_idx].argmax()
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [50]:
print(generate(matrix_mim, id2word_mim, word2id_mim).replace('<end>', '\n'))

а затем внезапно осветился изнутри что это хорошо это наташа борову на суде … предположим даже я сейчас более похудел и загорелось 
 но достаточно вам подарок 
 – вы что-нибудь простенькое 
 – выбежал маленький прихрамывающий иностранец мог победить другого 
 – ответил азазелло не зная форму выльется гнев бессилия 
 маргарита николаевна а другое возвышение для охраны горы сюда 
 – спросила маргарита стараясь за этой специальности то на корзинках с вас 
 – спросил – куда и все же вспомнился вчерашний сеанс кончился сеанс кончился приятным голосом профессор чувствуя себя повешенным на « что все равно также


In [51]:
print(generate(matrix_news, id2word_news, word2id_news).replace('<end>', '\n'))

данный прецедент цивилизованной стране 8 часов в приштине ранен 
 по их данные последнего времени 9.30 24 сентября назаседании совета директоров корпорации 
 как по оценкам милиции 
 трубников выразил предположение согласно которой мировым сообществом будет нарушений сообщил других бумаг так и 750 человек изъято 138,5 килограмма такого рода должна быть сняты с совместным проектом нацеленным на будущий состав буйнакского района 
 в момент готова к взрывам жилых домов на определенные плюсы и уголовный кодекс рф официальный представитель госдепартамента сша в составе департамента 
 генеральный секретарь нато ответил что его личное их сплавов 
 чаще по конституционному законодательству теперь инвалиды


## Beam Search

Выше мы попробовали два способа выбирать предсказания на основе имеющихся вероятностей - 1) брать самое вероятное и 2) семплировать согласно распределению. К этому можно добавлять еще много других настроек и со многими мы еще поработаем, когда дойдем до больших языковых моделей. Сейчас давайте разберем еще один алгоритм, который можно применять для улучшения генерации. Он называется beam search (поиск лучом? лучевой поиск?).

![](https://opennmt.net/OpenNMT/img/beam_search.png)

Идея тут в том, чтобы на каждом шаге генерировать несколько вариантов продолжений, а затем несколько вариантов и для каждого из предыдущих продолжений. Таким образом, получается дерево генерации, где каждый вариант на следующем шаге ветвится на несколько других вариантов. Чтобы дерево не разрасталось слишком сильно в beam search есть параметр, которым задает максимальное количество вариантов на каждом шаге. Если вариантов больше, то часть из них удаляется и больше не продолжается. Чтобы отранжировать варианты, для каждого из них расчитывается общая вероятность (всей последовательности!) и выбираются самые вероятные. Обратите внимание, что на картинке на каждом из шагов не более 5 вариантов, а некоторые не доживают до последнего шага.

При простой генерации по одному слову есть вероятность сделать неправильный выбор и закрыть возможность для других хороших продолжений. А beam search позволяет рассматривать сразу несколько вариантов и вероятность пойти не туда, значительно снижается. 

Давайте напишем функцию для генерации с помощью beam search

In [52]:
# сделаем класс чтобы хранить каждый из лучей
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score 

In [53]:

def generate_with_beam_search(matrix, id2word, word2id, n=100, max_beams=5, start='<start>'):
    # изначально у нас один луч с заданным началом (start по дефолту)
    initial_node = Beam(sequence=[start], score=np.log1p(0))
    beams = [initial_node]
    
    for i in range(n):
        # делаем n шагов генерации
        new_beams = []
        # на каждом шаге продолжаем каждый из имеющихся лучей
        for beam in beams:
            # лучи которые уже закончены не продолжаем (но и не удаляем)
            if beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue
            
            # наша языковая модель предсказывает на основе предыдущего слова
            # достанем его из beam.sequence
            last_id = word2id[beam.sequence[-1]]
            
            # посмотрим вероятности продолжений для предыдущего слова
            probas = matrix[last_id].toarray()[0]
            
            # возьмем топ самых вероятных продолжений
            top_idxs = probas.argsort()[:-(max_beams+1):-1]
            for top_id in top_idxs:
                # иногда вероятности будут нулевые, такое не добавляем
                if not probas[top_id]:
                    break
                
                # создадим новый луч на основе текущего и варианта продолжения
                new_sequence = beam.sequence + [id2word[top_id]]
                # скор каждого луча это произведение вероятностей (или сумма логарифмов)
                new_score = beam.score + np.log1p(probas[top_id])
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)
        # отсортируем лучи по скору и возьмем только топ max_beams
        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]
    
    # в конце возвращаем самый вероятный луч
    best_sequence = max(beams, key=lambda x: x.score).sequence

    
    return ' '.join(best_sequence)

In [54]:
print(generate_with_beam_search(matrix_news, id2word_news, word2id_news, start='куда'))

куда хинштейн предъявил вашингтону ультиматум предъявленный жителям а также отметил что в связи с 1 января 2000 года в том что в связи с 1 января 2000 года в том что в связи с 1 января 2000 года в том что в связи с 1 января 2000 года в том что в связи с 1 января 2000 года в том что в связи с 1 января 2000 года в том что в связи с 1 января 2000 года в том что в связи с 1 января 2000 года в том что в связи с 1 января 2000 года в том что


### Перплексия

Генерировать текст полностью требуется достаточно редко. И к тому же с этим более менее адекватно справляются только самые современные огромные модели. Гораздно более практичное применения языковой модели - выбрать наиболее вероятное продолжение уже введенной фразы. Вы все сталкивались с этим в своих телефонах. Если попытаться сгенировать текст только на основе предложенных слов, то получится не сильно лучше чем в текстах выше. Также языковую модель можно использовать, чтобы выбрать наиболее подходящее по контексту исправление опечатки и в этом случае совсем не важно, насколько красивые тексты она генерирует.

Но как тогда оценивать качество языковой модели? Для этого стандартно используется перплексия (на русский обычно не переводят). У перплексии есть теоретическое обоснование в теории информации и даже какая-то интерпретация, но они достаточно сложные и непонятные. На практике можно просто считать, что перплексия показывает насколько хорошо языковая модель предсказывает корпус. Чем она ниже, тем лучше.

Считается перплексия по вот такой формуле:


In [107]:
Image(url="https://i.ibb.co/Ph3sNMp/image.png",
     width=500, height=500)

Простыми словами - нам нужно расчитать вероятность текста (мы это уже научились делать выше) и возвести ее в степень (-1/N), где N это количество слов в тексте.

In [55]:
# Мы уже видели что произведение вероятностей можно заменить на экспоненту суммы логарифмов
# С возведением в степень тоже есть удобное правило - log(x^y) = y * log(x)
# можно заменить вот такую функцию (она ожидает вероятность)
# def perplexity(p, N):
#     return p**(-1/N) 


# на вот такую (результат должен совпадать)
# функция ожидает логарифм вероятности
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

Нам нужно немного изменить функцию для расчета вероятности, чтобы возвращать N

In [56]:
# функции возвращают лог (чтобы проверить с первой функцией можно добавить np.exp(prob))
def compute_joint_proba(text, word_probas):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)
    
    return prob, len(tokens)


def compute_join_proba_markov_assumption(text, word_counts, bigram_counts):
    prob = 0
    tokens = normalize(phrase)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>']):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)
    
    return prob, len(tokens)

У нас есть две функции для генерации вероятности последовательности. По сути каждая функция - это языковая модель. Первая - униграмная, а вторая биграмная.

Таким образом мы можем сравнить две языковые модели, расчитанные на одном корпусе.

In [63]:
phrase = 'Однажды весною, в час небывало жаркого заката, в Москве, на Патриарших прудах, появились два гражданина. '

In [64]:
perplexity(*compute_joint_proba(phrase, probas_mim))

4206.749964729248

In [65]:
perplexity(*compute_join_proba_markov_assumption(phrase, unigrams_mim, bigrams_mim))

24.716272983026247

Перплексия второй (биграмной модели) сильно меньше. Значит она лучше предсказывает корпус

**Со вторым текстом, который мы рассматривали до этого такое не сработает, потому что слишком много биграммов нет в словарях. Поэтому перплексия биграммной модели будет выше. 

In [66]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'

In [67]:
perplexity(*compute_joint_proba(phrase, probas_news))

10737.11899899257

In [68]:
perplexity(*compute_join_proba_markov_assumption(phrase, unigrams_news, bigrams_news))

12287.628005974075

Но лучше оценивать качество языковой модели не на 1 тексте, а сразу на целом корпусе. Мы можем посчитать перплексию на всех предложениях и усреднить, чтобы получить общую перплексию

Униграмная модель:

In [74]:
ps = []
for sent in sent_tokenize(news[:5000000]):
    prob, N = compute_joint_proba(sent, probas_news)
    if not N:
        continue
    ps.append(perplexity(prob, N))

In [75]:
np.mean(ps)

9847.645534186024

Биграмная модель:

In [76]:
ps = []
for sent in sent_tokenize(news[:5000000]):
    prob, N = compute_join_proba_markov_assumption(sent, unigrams_news, bigrams_news)
    if not N:
        continue
    ps.append(perplexity(prob, N))

In [77]:
np.mean(ps)

12287.628005974078